In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, roc_curve, roc_auc_score

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import GlobalMaxPooling1D

In [3]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils  import to_categorical

In [4]:
df=pd.read_csv("Appliance_4.csv")

In [5]:
df.head()

,Occupancy,Appliance ID,Sin(time),Cos(time),Sin(day),Cos(day),Power consumption,Normalized power,P(t)-P(t-1),P(t)-P(t+1),Class Label
0,1,4,0.00000,1.00000,0.0,1.0,655,0.136000,655,-657,0
1,1,4,0.25882,0.96593,0.0,1.0,1312,0.501000,657,487,0
2,1,4,0.50000,0.86603,0.0,1.0,825,0.230440,-487,284,0
3,1,4,0.70711,0.70711,0.0,1.0,541,0.072663,-284,-635,0
4,1,4,0.86603,0.50000,0.0,1.0,1176,0.425440,635,409,0


In [6]:
df.columns

Index(['Occupancy', 'Appliance ID', 'Sin(time)', 'Cos(time)', 'Sin(day)',
       'Cos(day)', 'Power consumption', 'Normalized power', 'P(t)-P(t-1)',
       'P(t)-P(t+1)', 'Class Label'],
      dtype='object')

In [7]:
shuffle(df)

,Occupancy,Appliance ID,Sin(time),Cos(time),Sin(day),Cos(day),Power consumption,Normalized power,P(t)-P(t-1),P(t)-P(t+1),Class Label
10423,0,4,0.96593,-0.25882,0.927540,0.37372,584,0.096551,-161,-779,4
1527,1,4,-0.70711,-0.70711,0.884070,0.46736,775,0.202660,-62,-19,0
8361,0,4,0.70711,-0.70711,-0.288480,0.95749,0,-0.227890,0,0,0
9031,0,4,0.96593,-0.25882,0.188230,0.98213,1251,0.467110,-10,394,4
5055,0,4,-0.70711,-0.70711,-0.455910,-0.89003,829,0.232660,-568,-56,4
...,...,...,...,...,...,...,...,...,...,...,...
16571,0,4,0.25882,-0.96593,-0.635430,0.77216,0,-0.227890,0,0,0
13946,0,4,0.50000,0.86603,-0.545240,-0.83828,673,0.146000,-395,-287,4
13295,0,4,-0.25882,0.96593,-0.094537,-0.99552,719,0.171550,-275,-295,4
13546,1,4,0.50000,-0.86603,-0.280230,-0.95993,597,0.103770,-831,-529,0


In [8]:
# Split the data into train and test sets
train_size = int(len(df)*0.8)
train, test = df.iloc[:train_size],df.iloc[train_size:]

In [9]:
# Split train and test sets into predictors (X) and outcome (y) variables
x_train = train.drop('Class Label', axis = 1)
y_train = train.loc[:,['Class Label']]
x_test = test.drop('Class Label', axis = 1)
y_test = test.loc[:,['Class Label']]

In [10]:
# Scale the X data 
scaler_x = MinMaxScaler(feature_range = (0,1))
input_scaler = scaler_x.fit(x_train)
train_x_norm = input_scaler.transform(x_train)
test_x_norm = input_scaler.transform(x_test)

In [11]:
# One-hot encode y data
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((14016, 10), (3504, 10), (14016, 5), (3504, 5))

In [13]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [14]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

In [15]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((14016, 10, 1), (3504, 10, 1), (14016, 5), (3504, 5))

In [16]:
verbose, epochs, batch_size = 1, 50, 32
n_timesteps, n_features = x_train.shape[1], x_train.shape[2]

In [17]:
# Define the model architecture
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(10, 1)))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(5, activation='softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 8, 16)             64        
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 16)             784       
                                                                 
 conv1d_2 (Conv1D)           (None, 4, 16)             784       
                                                                 
 global_max_pooling1d (Globa  (None, 16)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                1088      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                        

In [19]:
#compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
#fit the model
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/50
438/438 [==============================] - 1s 1ms/step - loss: 3.3089 - accuracy: 0.5657
Epoch 2/50
438/438 [==============================] - 0s 1ms/step - loss: 0.6012 - accuracy: 0.7075
Epoch 3/50
438/438 [==============================] - 0s 1ms/step - loss: 0.2308 - accuracy: 0.9045
Epoch 4/50
438/438 [==============================] - 0s 1ms/step - loss: 0.1729 - accuracy: 0.9233
Epoch 5/50
438/438 [==============================] - 0s 989us/step - loss: 0.1431 - accuracy: 0.9376
Epoch 6/50
438/438 [==============================] - 0s 995us/step - loss: 0.1383 - accuracy: 0.9399
Epoch 7/50
438/438 [==============================] - 0s 1ms/step - loss: 0.1008 - accuracy: 0.9580
Epoch 8/50
438/438 [==============================] - 0s 1ms/step - loss: 0.0820 - accuracy: 0.9646
Epoch 9/50
438/438 [==============================] - 0s 1ms/step - loss: 0.0914 - accuracy: 0.9628
Epoch 10/50
438/438 [==============================] - 0s 985us/step - loss: 0.0627 - accuracy: 

In [21]:
_, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=verbose)
#  return accuracy

110/110 [==============================] - 0s 713us/step - loss: 0.0091 - accuracy: 0.9986


In [22]:
accuracy

0.9985730648040771

In [23]:
prediction = model.predict(x_test)

110/110 [==============================] - 0s 656us/step


In [24]:
# prediction[1]

In [25]:
y_test

array([[0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [26]:
# prediction_NN = np.argmax(prediction[1], axis=-1)

In [27]:
prediction_NN=np.argmax(prediction, axis=-1)

In [28]:
prediction_NN

array([2, 1, 4, ..., 4, 2, 0], dtype=int64)

In [29]:
y_test1 = test.loc[:,['Class Label']]

In [30]:
y_test1=np.array(y_test1)

In [31]:
y_test1

array([[2],
       [1],
       [4],
       ...,
       [4],
       [2],
       [0]], dtype=int64)

In [32]:
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test1, prediction_NN))

Accuracy =  0.9985730593607306


In [33]:
#Confusion Matrix - verify accuracy of each class
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test1, prediction_NN)
print(cm)

[[1711    0    0    4    0]
 [   0  185    0    0    0]
 [   0    0  186    0    0]
 [   0    0    0  343    1]
 [   0    0    0    0 1074]]


In [34]:
from sklearn.metrics import classification_report

# print(class_labels)
target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4']
print(classification_report(y_test1, prediction_NN, target_names=target_names,digits=3))

              precision    recall  f1-score   support

     Class 0      1.000     0.998     0.999      1715
     Class 1      1.000     1.000     1.000       185
     Class 2      1.000     1.000     1.000       186
     Class 3      0.988     0.997     0.993       344
     Class 4      0.999     1.000     1.000      1074

    accuracy                          0.999      3504
   macro avg      0.998     0.999     0.998      3504
weighted avg      0.999     0.999     0.999      3504



In [35]:
# !pip install mlxtend

In [36]:
# from mlxtend.plotting import plot_confusion_matrix

In [37]:
# class_dict = {0: 'Class 0',
#               1: 'Class 1',
#               2: 'Class 2',
#               3: 'Class 3',
#               4: 'Class 4'}

# multiclass = np.array([[9926,   67,   19,   56,    0],
#                        [178,  855,    0,   41,   40],
#                        [268,    0,  848,    0,    0],
#                        [27,   16,    0, 2326,   15],
#                        [0,    1,    0,   36, 6305]])

# fig, ax = plot_confusion_matrix(
#     conf_mat=multiclass,
#     class_names=class_dict.values(),show_absolute=False,show_normed=True,colorbar=True)